In [10]:
import requests
import os

In [11]:
def upload_pdf(pdf_path, api_key):
    files = [
        ('file', ('file', open(pdf_path, 'rb'), 'application/octet-stream'))
    ]
    headers = {
        'x-api-key': api_key
    }
    response = requests.post('https://api.chatpdf.com/v1/sources/add-file', headers=headers, files=files)

    if response.status_code == 200:
        return response.json()['sourceId']
    else:
        raise Exception(f"Failed to upload PDF: {response.status_code} - {response.text}")





In [12]:
def read_sources_from_file(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]

In [13]:
def write_sources_to_file(source_ids, file_path):
    with open(file_path, 'w') as file:
        for source_id in source_ids:
            file.write(f"{source_id}\n")

In [15]:
def query_pdf(source_id, query, api_key):
    headers = {
        'x-api-key': api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        'sourceId': source_id,
        'messages': [
        {
            'role': "user",
            'content': query,
        }
    ]
    }
    response = requests.post('https://api.chatpdf.com/v1/chats/message', headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()['content']
    else:
        raise Exception(f"Failed to query PDF: {response.status_code} - {response.text}")


In [16]:
if __name__ == "__main__":
    
    api_key = 'sec_xJ86UNfF83CywriPTtJjWh9X6zF9YKKc'

    pdf_paths = [
        r'C:\Users\wasam\OneDrive\Desktop\Summer\PDFs\WordPressManual.pdf',
        r'C:\Users\wasam\OneDrive\Desktop\Summer\PDFs\wordpress_tutorial.pdf',
        r'C:\Users\wasam\OneDrive\Desktop\Summer\PDFs\WordPress-User-Guide-Administrators.pdf'
    ]

    file_path = 'uploaded_pdf.txt'
    
    try:
        if os.path.exists(file_path):
            # Read source IDs from the file
            source_ids = read_sources_from_file(file_path)
            print("Source IDs read from file:")
            for source_id in source_ids:
                print(f"Source ID: {source_id}")
        
        else:
            # Upload PDFs and get source IDs
            source_ids = []
            for pdf_path in pdf_paths:
                source_id = upload_pdf(pdf_path, api_key)
                source_ids.append(source_id)
                print(f'Uploaded {pdf_path}, Source ID: {source_id}')

            # Write source IDs to the file
            write_sources_to_file(source_ids, file_path)
            print(f"Source IDs written to {file_path}")

        while True:
            user_input = input("You: ")
            print("You: ",user_input)
            if user_input.lower() in ["exit", "quit","bye"]:
                print("Exited")
                break

            answer_found = False

            for source_id in source_ids:
                try:
                    
                    # Query the PDF using the source ID
                    response = query_pdf(source_id, user_input, api_key)
                    if response:  # Assuming a non-empty response means an answer was found
                        print()
                        print(f"Bot: {response}")
                        answer_found = True
                        break
                except Exception as query_exception:
                    print(f"Error querying PDF (Source ID: {source_id}): {query_exception}")

            if not answer_found:
                print("No relevant information found in any of the PDFs.")

    except Exception as e:
        print(f"Error uploading PDF: {e}")


Source IDs read from file:
Source ID: src_XIVwUryBcHmW3KwcYLJnd
Source ID: src_w55IwocDiuntzkvrVBqax
Source ID: src_Wrq8DJOOKtm9jBZNvMrgH
You:  how to intsall word press. give me step by step instuctions

Bot: To install WordPress, follow these step-by-step instructions:

1. Go to your WordPress Dashboard area and click on Plugins » Add New.
2. Find the plugin by typing the plugin name or the functionality you are looking for.
3. Click the Install Now button to install it for your site.
4. WordPress will download and install the plugin for you. After this, you will see a success message with a link to Activate the plugin or Return to the plugin installer.

If you need further assistance or have any questions, feel free to ask!
You:  bye
Exited
